<a href="https://colab.research.google.com/github/Aurelio-Naufal/KI_Task_Aurelio-Naufal-Effendy/blob/main/MidtermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Torchvision and OpenCV module
%%time
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install opencv-python-headless
!pip install matplotlib

# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Install the required dependencies
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 27.24 MiB/s, done.
Resolving deltas: 100% (11695/11695), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.5/877.5 kB 16.6 MB/s eta 0:00:00
CPU times: user 124 ms, sys: 21 ms, total: 145 ms
Wall time: 14.6 s


In [2]:
#Import Necessary Module
import numpy as np
import pandas as pd
import cv2, torch
from tqdm.auto import tqdm
from google.colab import drive, files

from IPython.display import Video, HTML
from base64 import b64encode

In [3]:
#Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
video_path = '/content/drive/MyDrive/Tugas/Long Shot of Busy Traffic Driving In Downtown Jakarta.mp4'

In [5]:
import torch

# Load YOLOv5 model (v5.0)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s',pretrained=True)
# Use Colab GPU
model.to('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-10-25 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 111MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [6]:
# Set confidence and IoU thresholds
conf_threshold = 0.25  # Confidence threshold
iou_threshold = 0.6   # IoU threshold

In [7]:
import cv2
import torch
import pandas as pd
from google.colab.patches import cv2_imshow
from torchvision.ops import nms  # Import NMS from torchvision


# Load the video
video_capture = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not video_capture.isOpened():
    print("Error: Could not open video.")
    exit()

# Dictionary to store random colors for each class
color_palette = [
    (255, 0, 0),    # Red
    (0, 255, 0),    # Green
    (0, 0, 255),    # Blue
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
    (128, 0, 128),  # Purple
    (255, 165, 0),  # Orange
    (0, 128, 128),  # Teal
    (128, 128, 0)   # Olive
]

colors = {}

# Function to generate a fixed color for bounding box
def get_fixed_color(label):
    if label not in colors:
        colors[label] = color_palette[len(colors) % len(color_palette)]
    return colors[label]


# Get video properties
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))

# Initialize VideoWriter to save the output video with bounding boxes
output_video = cv2.VideoWriter('output_with_detections.mp4',
                               cv2.VideoWriter_fourcc(*'mp4v'),
                               fps,
                               (frame_width, frame_height))

# Initialize an empty list to store detection results
detection_results = []

frame_index = 0  # Track frame number

while True:
    ret, frame = video_capture.read()

    if not ret:  # Exit if no frame is returned
        print("End of video or error reading frame.")
        break

    # Check if frame is valid
    if frame is None:
        print("Warning: Received an empty frame.")
        continue

    # Convert frame to RGB for YOLO
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # YOLOv5 inference on the frame
    results = model(frame_rgb)

    # Extract bounding boxes, confidence scores, and class labels
    bbox_data = results.xyxy[0].cpu().numpy()  # Get boxes, confidences, and labels
    boxes = bbox_data[:, :4]  # Bounding box coordinates
    scores = bbox_data[:, 4]  # Confidence scores
    labels = bbox_data[:, 5]  # Class labels

    # Convert to tensors for NMS (Non Maximum Supression)
    boxes_tensor = torch.tensor(boxes)
    scores_tensor = torch.tensor(scores)

    # Apply NMS (Non Maximum Supression)
    indices = nms(boxes_tensor, scores_tensor, iou_threshold)
    filtered_boxes = bbox_data[indices]  # Filter boxes based on NMS

    # Process each detection in the current frame
    for bbox in filtered_boxes:
        x1, y1, x2, y2, confidence, cls = bbox

        if confidence.item() >= conf_threshold:  # Confidence threshold check
          label = model.names[int(cls)]  # Get class name

          # add different color on each label
          color = get_fixed_color(label)

          # Draw bounding box and label on the frame
          cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
          cv2.putText(frame, f'{label} {confidence:.2f}', (int(x1), int(y1) - 10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

          # Normalize bounding box coordinates
          height, width, _ = frame.shape
          x1_norm = x1 / width
          y1_norm = y1 / height
          x2_norm = x2 / width
          y2_norm = y2 / height

          # Append the detection result as a dictionary
          detection_results.append({
              'frame': frame_index,
              'x1': x1_norm,
              'y1': y1_norm,
              'x2': x2_norm,
              'y2': y2_norm,
              'confidence': confidence,
              'class': label
          })

    frame_index += 1
    output_video.write(frame)

# Release resources
video_capture.release()
output_video.release()

print("Object detection data saved to DataFrame.")

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
WARNING ⚠️ NMS time limit 0.550s exceeded
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/co

End of video or error reading frame.
Object detection data saved to DataFrame.


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [8]:
# Convert the list of detection results to a pandas DataFrame
df_detections = pd.DataFrame(detection_results)

# Display the DataFrame
print(df_detections)

# Optionally, save the DataFrame to a CSV file
df_detections.to_csv('object_detection_results.csv', index=False)

      frame        x1        y1        x2        y2  confidence       class
0         0  0.169345  0.380929  0.266616  0.596982    0.900624         bus
1         0  0.275954  0.571143  0.358001  0.724188    0.844074         car
2         0  0.277093  0.301296  0.338894  0.433332    0.781674         bus
3         0  0.349096  0.457135  0.410967  0.560766    0.757206         car
4         0  0.562616  0.440228  0.639109  0.639524    0.691132       truck
...     ...       ...       ...       ...       ...         ...         ...
7090    283  0.851760  0.591180  0.884776  0.639708    0.334934       bench
7091    283  0.534678  0.407872  0.557523  0.464920    0.298774  motorcycle
7092    283  0.541005  0.592192  0.565059  0.655735    0.272370      person
7093    283  0.545995  0.438343  0.569423  0.512651    0.272172  motorcycle
7094    283  0.496581  0.360563  0.512393  0.419493    0.269802  motorcycle

[7095 rows x 7 columns]


In [9]:
from google.colab import files

# Download the video to your local machine
files.download('output_with_detections.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>